In [1]:
# mount drive https://datascience.stackexchange.com/questions/29480/uploading-images-folder-from-my-system-into-google-colab
# login with your google account and type authorization code to mount on your googlbie drive.
from google.colab import drive
drive.mount('/gdrive')
root = '/gdrive/My Drive/CS492I/project'

Mounted at /gdrive


## 1. Collect CodeSearchNet Repositories

In [ ]:
import json

import pandas as pd
from pathlib import Path
pd.set_option('max_colwidth',300)
from pprint import pprint

In [ ]:
!wget https://s3.amazonaws.com/code-search-net/CodeSearchNet/v2/python.zip
!mkdir CodeSearchNet
!unzip python.zip -d CodeSearchNet

--2021-12-03 07:32:59--  https://s3.amazonaws.com/code-search-net/CodeSearchNet/v2/python.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.69.238
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.69.238|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 940909997 (897M) [application/zip]
Saving to: ‘python.zip’

python.zip          100%[===================>] 897.32M  65.4MB/s    in 13s     

2021-12-03 07:33:12 (68.1 MB/s) - ‘python.zip’ saved [940909997/940909997]

Archive:  python.zip
   creating: CodeSearchNet/python/
   creating: CodeSearchNet/python/final/
   creating: CodeSearchNet/python/final/jsonl/
   creating: CodeSearchNet/python/final/jsonl/train/
  inflating: CodeSearchNet/python/final/jsonl/train/python_train_9.jsonl.gz  
  inflating: CodeSearchNet/python/final/jsonl/train/python_train_12.jsonl.gz  
  inflating: CodeSearchNet/python/final/jsonl/train/python_train_10.jsonl.gz  
  inflating: CodeSearchNet/python/final/jsonl/train/pyt

In [ ]:
python_files = sorted(Path('CodeSearchNet/python').glob('**/*.gz'))
print(python_files)

[PosixPath('CodeSearchNet/python/final/jsonl/test/python_test_0.jsonl.gz'), PosixPath('CodeSearchNet/python/final/jsonl/train/python_train_0.jsonl.gz'), PosixPath('CodeSearchNet/python/final/jsonl/train/python_train_1.jsonl.gz'), PosixPath('CodeSearchNet/python/final/jsonl/train/python_train_10.jsonl.gz'), PosixPath('CodeSearchNet/python/final/jsonl/train/python_train_11.jsonl.gz'), PosixPath('CodeSearchNet/python/final/jsonl/train/python_train_12.jsonl.gz'), PosixPath('CodeSearchNet/python/final/jsonl/train/python_train_13.jsonl.gz'), PosixPath('CodeSearchNet/python/final/jsonl/train/python_train_2.jsonl.gz'), PosixPath('CodeSearchNet/python/final/jsonl/train/python_train_3.jsonl.gz'), PosixPath('CodeSearchNet/python/final/jsonl/train/python_train_4.jsonl.gz'), PosixPath('CodeSearchNet/python/final/jsonl/train/python_train_5.jsonl.gz'), PosixPath('CodeSearchNet/python/final/jsonl/train/python_train_6.jsonl.gz'), PosixPath('CodeSearchNet/python/final/jsonl/train/python_train_7.jsonl.gz

In [ ]:
columns_long_list = ['repo', 'path', 'url', 'code', 
                     'code_tokens', 'docstring', 'docstring_tokens', 
                     'language', 'partition']

def jsonl_list_to_dataframe(file_list, columns=columns_long_list):
    """Load a list of jsonl.gz files into a pandas DataFrame."""
    return pd.concat([pd.read_json(f, 
                                   orient='records', 
                                   compression='gzip',
                                   lines=True)[columns] 
                      for f in file_list], sort=False)

In [ ]:
pydf = jsonl_list_to_dataframe(python_files, columns=['repo'])
pydf = pydf.drop_duplicates().reset_index(drop=True)
pydf.to_pickle(f"{root}/repos.pkl")

## 2. Collect diff and commits

In [2]:
!pip install pydriller
!pip install pandas

     |████████████████████████████████| 65 kB 2.1 MB/s 
     |████████████████████████████████| 180 kB 11.5 MB/s 
     |████████████████████████████████| 64 kB 3.0 MB/s 
     |████████████████████████████████| 63 kB 1.1 MB/s 


In [3]:
from pydriller import *
import pandas as pd
from nltk.tokenize import regexp_tokenize
import re
from itertools import chain
import json

In [4]:
repodf = pd.read_pickle(f"{root}/repos.pkl")
print(repodf.shape)
print(repodf)

(13590, 1)
                                 repo
0                     soimort/you-get
1                      apache/airflow
2                      pytorch/vision
3                      asciimoo/searx
4              tensorflow/probability
...                               ...
13585         praekelt/python-ambient
13586                 zenreach/py-era
13587  TakesxiSximada/custom_settings
13588            openpermissions/bass
13589               xnuinside/clifier

[13590 rows x 1 columns]


In [6]:
def basic_filter(message):
    return message.split("\n", 1)[0].strip()

# Remove [label] in front of commit if exists
def label_filter(message):
    if (message.startswith('[')):
        end_bracket_index = message.find(']')
        if (end_bracket_index == -1):
            return None
        return message[:end_bracket_index+1]
    return message

def camel_case_split(str):
    return re.findall(r'[A-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))', str)

def case_splitter(token):
    return list(map(lambda x: x.lower(), camel_case_split(token[0].upper() + token[1:]) if token[0].isalpha() else [token]))


def split_by_quote(diff):
    text_list = [(0,0)]
    quote_state = 0 #["out","single","double","Single","Double","quote"]
    i=0
    while i<len(diff):
      if diff[i] == "\\":i+=1
      elif (diff[i] == "\n" and quote_state == 5) or (diff[i] == '#' and quote_state == 0):
        text_list.append((quote_state,i))
        quote_state = abs(5-quote_state)
      elif diff[i] == '"':
        if i+2<len(diff) and diff[i:i+3]=='"""' and (quote_state == 0 or quote_state ==4):
          text_list.append((quote_state,i+max(quote_state-1,0)))
          quote_state = abs(4-quote_state)
          i+=2
        elif quote_state == 0 or quote_state == 2:
          text_list.append((quote_state,i+max(quote_state-1,0)))
          quote_state = abs(2-quote_state)
      elif diff[i] == "'":
        if i+2<len(diff) and diff[i:i+3] == "'''" and (quote_state == 0 or quote_state == 3):
          text_list.append((quote_state,i+quote_state))
          quote_state = abs(3-quote_state)
          i+=2
        elif quote_state == 0 or quote_state == 1:
          text_list.append((quote_state,i+quote_state))
          quote_state = abs(1-quote_state)
      i+=1
    text_list.append((quote_state,i))
    return [(text_list[i][0]==0,diff[text_list[i-1][1]:text_list[i][1]]) for i in range(1,len(text_list))]
  

token_regex = r"""(?x)
<(?:add|del)>   #Filtered eariler
|(?:[-+*/^&~|=%!<>@?$][\s]*)+     #Sequence of symbols
|[\n]                    #Change row
|[a-zA-Z]+               #General text
|[0-9]+                  #Number
"""
string_regex = r"""(?x)
<(?:add|del)>
|[\n]
|[a-zA-Z]+
"""
def diff_tokenizer(diff_text):
    diff = '\n'.join(map(lambda x: x[1], filter(lambda y: y[0] % 2 == 0, enumerate(diff_text.split("@@")))))
    diff = diff.replace('\n+', '\n<add>').replace('\n-', '\n<del>')
    diff = re.sub(r"(?:\n[ \t\r\f\v]*)+","\n",diff) # Join continuous row change
    quote_split = split_by_quote(diff)
    token_initial = chain.from_iterable(map(lambda a: regexp_tokenize(a[1],(token_regex if a[0] else string_regex)), quote_split))
    token_camel_case_split = chain.from_iterable(map(case_splitter, token_initial))
    return token_camel_case_split


def parse_repo_commits(repo_name, commit_limit=50):
    data = []
    commit_count = 0
    for commit in Repository(
        f"https://github.com/{repo_name}",
        only_modifications_with_file_types=[".py"],
        only_no_merge=True,
        order='reverse'
    ).traverse_commits():
        if (commit_count >= commit_limit): break
        line = basic_filter(commit.msg)
        line = label_filter(line)
        if (line is None):
            continue
        
        if line.count("`")%2 == 1:
            continue
        
        backtick_regex = r"`[^`]+`|[^`]+"
        line_divided_by_backtick = re.findall(backtick_regex,line)
        tokens = chain.from_iterable(map(lambda text: regexp_tokenize(text,(token_regex if text[0] == '`' else string_regex)),line_divided_by_backtick))
        
        commit_tokens = list(chain.from_iterable(map(case_splitter, tokens)))

        if (len(commit_tokens) < 3 or len(commit_tokens) > 30):
            continue
        if ("version" in commit_tokens):
            continue

        # Check if changed files are python
        file_failed = False
        count_diff_lines = 0

        for mf in commit.modified_files:
            if (not mf.filename.endswith(".py")):
                file_failed = True
                break
            count_diff_lines += mf.added_lines + mf.deleted_lines
        
        if (file_failed):
            continue
        
        if (count_diff_lines > 50):
            continue
        
        # Create diff tokens
        diff_tokens = []
        for f in commit.modified_files:
          diff_tokens.append (['<file>'])
          diff_tokens.append(diff_tokenizer(f.diff))

        diff_whole_tokens = list(chain.from_iterable(diff_tokens))

        data.append([repo_name, commit.hash, json.dumps(commit_tokens), json.dumps(diff_whole_tokens)])
        commit_count += 1

    return pd.DataFrame(data, columns=["repo", "hash", "commit_messsage", "diff"])

In [7]:
import time
start = time.time()
#df = parse_repo_commits("soimort/you-get")
df = parse_repo_commits("tensorflow/probability")
end = time.time()
print(end - start)
df.head(10)

20.40401601791382


,repo,hash,commit_messsage,diff
0,tensorflow/probability,c2299ff88eaf1c3f29dc48ce5a80c3db8bc8625d,"[""ensure"", ""owens"", ""t"", ""special"", ""function""...","[""<file>"", ""\n"", ""def"", ""owens"", ""t"", ""method""..."
1,tensorflow/probability,f5541be80d2e118a3875ec795285927f5580465a,"[""always"", ""return"", ""kalman"", ""filter"", ""resu...","[""<file>"", ""\n"", ""from"", ""tensorflow"", ""python..."
2,tensorflow/probability,9990d6105450b43e2169968a397c0ce8d98598cb,"[""improves"", ""error"", ""message"", ""in"", ""dtype""...","[""<file>"", ""\n"", ""def"", ""base"", ""equal"", ""a"", ..."
3,tensorflow/probability,ae7117f37ac441bc7a888167ea23e5e620c5bcde,"[""hack"", ""convert"", ""to"", ""nested"", ""tensor"", ...","[""<file>"", ""\n"", ""class"", ""bijector"", ""caching..."
4,tensorflow/probability,b305c9c66ae1b7f3f2446eeb9422fcff18eeb0e3,"[""restrain"", ""hypothesis"", ""from"", ""generating...","[""<file>"", ""\n"", ""def"", ""fix"", ""triangular"", ""..."
5,tensorflow/probability,16c382e6ca2fd897c8023a4169eeab9b3575c134,"[""re"", ""narrow"", ""numerical"", ""tolerances"", ""p...","[""<file>"", ""\n"", ""autovectorization"", ""rtol"", ..."
6,tensorflow/probability,8e03d0ce95154f0e4b495c69ed21e7b58641d705,"[""relax"", ""precision"", ""of"", ""truncated"", ""nor...","[""<file>"", ""\n"", ""event"", ""space"", ""bijector"",..."
7,tensorflow/probability,40b18076b53d7c6e22e1fa5f95d805be0cfa0cc1,"[""enable"", ""test"", ""the"", ""use"", ""of"", ""pmax"",...","[""<file>"", ""\n"", ""def"", ""windowed"", ""adaptive""..."
8,tensorflow/probability,bb15d65fafb9311475878f518a328c8f9bdcf31e,"[""replace"", ""value"", ""in"", ""simple"", ""step"", ""...","[""<file>"", ""\n"", ""class"", ""simple"", ""step"", ""s..."
9,tensorflow/probability,fe99538609c82abed2787d5cb3ef6ff1e23af673,"[""fix"", ""some"", ""documentation"", ""typos""]","[""<file>"", ""\n"", ""class"", ""bijector"", ""cache"",..."


# 3. Create whole dataset via Multiprocessing

In [13]:
import multiprocessing

data = []

def f(repo):
    start = time.time()
    df = parse_repo_commits(repo)
    #df.to_pickle(f"./repos/{repo.replace('/', '+')}.pkl")
    end = time.time()
    print(f"{repo} took {end-start} seconds")
    return df

pool = multiprocessing.Pool()
l = len(repodf['repo'])
outputs = pool.map(f, repodf['repo'][:l/2])
pd.concat(outputs).to_pickle(f"{root}/data.pkl")

jdodds/feather took 1.9959485530853271 seconds
soimort/you-get took 12.874940633773804 seconds
tamasgal/km3pipe took 22.724501848220825 seconds
IRC-SPHERE/HyperStream took 13.98513150215149 seconds
vslutov/turingmarkov took 1.3664534091949463 seconds
apache/airflow took 30.607398748397827 seconds
ioos/pyoos took 4.227296352386475 seconds
developmentseed/sentinel-s3 took 5.675388813018799 seconds
PrefPy/prefpy took 6.754190683364868 seconds
astooke/gtimer took 4.003330707550049 seconds
NaPs/Kolekto took 4.851297855377197 seconds
pytorch/vision took 23.18395233154297 seconds
rorychatterton/Terraform-Enterprise-2-Python-SDK took 2.661846160888672 seconds
wtsi-hgi/consul-lock took 7.079454183578491 seconds
vaidik/incoming took 2.1253509521484375 seconds
LeadPages/gcloud_requests took 7.498109817504883 seconds
materials-data-facility/toolbox took 7.162588596343994 seconds
tapilab/brandelion took 2.934922218322754 seconds
thebigmunch/google-music-utils took 4.676528692245483 seconds
asciimoo

Process ForkPoolWorker-10:
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/usr/lib/python3.7/multiprocessing/pool.py", line 44, in mapstar
    return list(map(*args))
  File "<ipython-input-13-12b3210429a7>", line 7, in f
    df = parse_repo_commits(repo)
  File "<ipython-input-6-7151f863dbcc>", line 79, in parse_repo_commits
    order='reverse'


KeyboardInterrupt: ignored

Process ForkPoolWorker-9:
  File "/usr/local/lib/python3.7/dist-packages/pydriller/repository.py", line 233, in traverse_commits
    for commit in job.result():
  File "/usr/local/lib/python3.7/dist-packages/pydriller/repository.py", line 240, in _iter_commits
    if self._conf.is_commit_filtered(commit):
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pydriller/utils/conf.py", line 267, in is_commit_filtered
    if not self._has_modification_with_file_type(commit):
  File "/usr/local/lib/python3.7/dist-packages/pydriller/utils/conf.py", line 282, in _has_modification_with_file_type
    for mod in commit.modified_files:
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.7/dist-packages/pydriller/domain/commit.py", line 668, in modified_files
    self._modificatio

In [11]:
df = pd.read_pickle(f"{root}/data.pkl")
df.head(3)

,repo,hash,commit_messsage,diff
0,soimort/you-get,5a12118c836b0b9a7a2f8bdbed25be9aa2ca7831,"[""add"", ""param"", ""postfix"", ""to"", ""postfix"", ""...","[""<file>"", ""\n"", ""output"", ""filename"", ""= "", ""..."
1,soimort/you-get,3f47a215bf5f73e73cd7aa607b340a7b68b9bd3d,"[""fix"", ""the"", ""name"", ""m"", ""u"", ""is"", ""not"", ...","[""<file>"", ""\n"", ""cookies"", ""= "", ""none"", ""\n""..."
2,soimort/you-get,424b555500e65da0533d369a80562bb93907ec9a,"[""fix"", ""zhihu"", ""extractor"", ""comment"", ""ld""]","[""<file>"", ""\n"", ""def"", ""zhihu"", ""download"", ""..."
